In [1]:
suppressMessages(library("CARD", quietly = T))
suppressMessages(library("ggplot2", quietly = T))
suppressMessages(library("reshape2", quietly = T))
suppressMessages(library("gtools", quietly = T))
suppressMessages(library("scatterpie", quietly = T))
suppressMessages(library("glue", quietly = T))

Warning message:
“replacing previous import ‘RcppML::nmf’ by ‘NMF::nmf’ when loading ‘CARD’”


In [2]:
# devtools::install_github('xuranw/MuSiC')

In [15]:
# load spatial data
key = "ND"
subId = "global_"
# subId = ""
inputDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDInputs/"

sptPath = glue("{inputDir}{key}_{subId}spt.csv")
spt.df <- read.csv(sptPath)
rownames(spt.df) <- spt.df[,1]
spt.df <- spt.df[,-1]
spt <- as.matrix(spt.df)
dim(spt)

# load spatial coordinates
xyPath = glue("{inputDir}{key}_{subId}coords.csv")
xy.df <- read.csv(xyPath)
rownames(xy.df) <- xy.df[,1]
xy.df <- xy.df[,-1]
dim(xy.df)

# load the rna seq data
rnaPath = glue("{inputDir}_{subId}rna.csv")
rna.df <- read.csv(rnaPath)
rownames(rna.df) <- rna.df[,1]
rna.df <- rna.df[,-1]
rna <- as.matrix(rna.df)
dim(rna)

# load the annotations
metaPath = glue("{inputDir}_{subId}clusters.csv")
meta.df <- read.csv(metaPath)
rownames(meta.df) <- meta.df$cellId
dim(meta.df)

[1] 31053  2036

[1] 2036    2

[1] 31053 13820

[1] 13820     4

In [16]:
CARD_obj = createCARDObject(
	sc_count = rna,
	sc_meta = meta.df,
	spatial_count = spt,
	spatial_location = xy.df,
	ct.varname = "cellType",
	ct.select = unique(meta.df$cellType),
	sample.varname = "sampleInfo",
	minCountGene = 10,
	minCountSpot = 20)

## QC on scRNASeq dataset! ...
## QC on spatially-resolved dataset! ...


In [17]:
CARD_obj = suppressMessages(CARD_deconvolution(CARD_object = CARD_obj))

## create reference matrix from scRNASeq...


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.0 GiB”


## Select Informative Genes! ...
## Deconvolution Starts! ...
## Deconvolution Finish! ...


In [18]:
props.df <- as.data.frame(CARD_obj@Proportion_CARD)

outDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDOuputs/"
outpath = glue("{outDir}{key}_{subId}CARD.csv")

write.csv(props.df, 
          outpath, 
          row.names=TRUE)

dim(props.df)

head(props.df)

[1] 2034   11

,B cells,Mac1,Mac3,Monocytes,T cells,NK cells,Stromal cells,Dendritic cells,Mac4,Mac2,Mac5
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAACAAGTATCTCCCA.1_ND,0.018690515,0.0094096631,1.073706e-02,0.07078242,0.04029819,0.07738866,0.6453321,0.0152807955,0.04551563,0.03935429,0.027210711
AAACAGCTTTCAGAAG.1_ND,0.008378193,0.0057003600,3.803781e-02,0.08993247,0.04861780,0.10398279,0.5750206,0.0467584445,0.02002660,0.03545058,0.028094348
AAACAGGGTCTATATT.1_ND,0.024324824,0.0107611417,2.432328e-02,0.11760583,0.07299279,0.12109297,0.4783830,0.0266079288,0.04205138,0.04825837,0.033598472
AAACATTTCCCGGATT.1_ND,0.023002457,0.0147036170,4.418104e-02,0.08701925,0.05353735,0.08392241,0.4659584,0.0532225554,0.08775848,0.03745193,0.049242491
AAACCCGAACGAAATC.1_ND,0.021953636,0.0037842781,1.239947e-03,0.03802028,0.02638629,0.06136367,0.7751703,0.0037251778,0.02310603,0.02591410,0.019336275
AAACCGGGTAGGTACC.1_ND,0.003432934,0.0006376751,3.671747e-05,0.02798927,0.01161905,0.03501702,0.8871449,0.0000331209,0.01217640,0.01429988,0.007613053


In [7]:
# # pieViz <- function(proportion, spatial_location){
# #     res_CARD = as.data.frame(proportion)
# #     res_CARD = res_CARD[,mixedsort(colnames(res_CARD))]
# #     location = as.data.frame(spatial_location) 
# #     colors = colorRampPalette(c("Blue", "Red"))(ncol(res_CARD))
# #     data = cbind(res_CARD,location)
# #     ct.select = colnames(res_CARD)
# #     p = ggplot() + 
# #         geom_scatterpie(aes(x=x, y=y, r = 1), 
# #                         data=data,
# #                         cols=ct.select,color=NA) + 
# #         coord_fixed(ratio = 1) + 
# #         scale_fill_manual(values = colors) 
# #     return(p)
    
# # }

# p <- CARD.visualize.pie(proportion = CARD_obj@Proportion_CARD,
#                         spatial_location = CARD_obj@spatial_location)
                   
# ggsave("cardFigures/mixtures.png",
#        width = 50, 
#        height = 50, 
#        units = "cm")

# print('done')

In [8]:
visCells <- function(proportion, spatial_location, ct.visualize, colors, NumCols, pntSize){
    res_CARD = as.data.frame(proportion)
    res_CARD = res_CARD[,order(colnames(res_CARD))]
    location = as.data.frame(spatial_location)
    if(sum(rownames(res_CARD)==rownames(location))!= nrow(res_CARD)){
    stop("The rownames of proportion data does not match with the rownames of spatial location data")
    }
    ct.select = ct.visualize
    res_CARD = res_CARD[,ct.select]
    res_CARD_scale = as.data.frame(apply(res_CARD,2,function(x){
    (x - min(x)) / (max(x) - min(x))
    } ))
    res_CARD_scale$x = as.numeric(location$x)
    res_CARD_scale$y = as.numeric(location$y)
    mData = melt(res_CARD_scale,id.vars = c("x","y"))
    colnames(mData)[3] <- "Cell_Type"
    b = c(0,1)
    
    p = ggplot(mData, aes(x, y))  +
    geom_point(aes(colour = value), size = pntSize) +
    scale_color_gradientn(colours = colors) + 
    scale_y_reverse() +
    facet_wrap(~Cell_Type,ncol = NumCols)+ 
    coord_fixed()+
    theme(axis.text = element_blank(),
          axis.ticks = element_blank(),
          axis.title = element_blank(),
          panel.background = element_blank(),
          plot.background = element_blank(),
          panel.grid.major = element_blank(), 
          panel.grid.minor = element_blank(),
         )
    return(p)
}

p <- visCells(CARD_obj@Proportion_CARD,
         CARD_obj@spatial_location,
         unique(meta.df$cellType),
         c("lightblue","lightyellow","red"),
         4, 
         0.1)

figDir = "cardFigures/"
outpath = glue("{figDir}{key}_{subId}spots.png")
ggsave(outpath,
       width = 20, 
       height = 20, 
       units = "cm")

print('done')

[1] "done"


In [9]:
CARD_obj = CARD.imputation(CARD_obj,
                           NumGrids = 10000,
                           ineibor = 21,
                           exclude = NULL)

location_imputation = cbind.data.frame(x=as.numeric(sapply(strsplit(rownames(CARD_obj@refined_prop),split="x"),"[",1)),
                                       y=as.numeric(sapply(strsplit(rownames(CARD_obj@refined_prop),split="x"),"[",2)))

rownames(location_imputation) = rownames(CARD_obj@refined_prop)

p <-visCells(
	CARD_obj@refined_prop,                         
	location_imputation,            
	unique(meta.df$cellType),                    
	c("lightblue","lightyellow","red"),    
	NumCols = 4,
    pntSize = 0.1)     

outpath = glue("{figDir}{key}_{subId}imputed.png")
ggsave(outpath,
       width = 20, 
       height = 20, 
       units = "cm")

print('done')

## The rownames of locations are matched ...
## Make grids on new spatial locations ...
[1] "done"


In [10]:
df <- as.data.frame(CARD_obj@refined_prop)
head(df)

head(location_imputation)

,Mac5,Monocytes,T cells,Mac4,Mac1,NK cells,Dendritic cells,Stromal cells,B cells,Mac2,Mac3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
331x476,0.3034526,0.07928022,0.04397935,0.03105100,0.06866088,0.09012360,0.02800539,0.2493845,0.04629264,0.02963406,0.03013585
331x487,0.3052533,0.07976759,0.04360004,0.03112132,0.06632515,0.09038673,0.02831381,0.2493849,0.04576633,0.02966085,0.03041997
320x498,0.3060100,0.07981767,0.04351404,0.03110309,0.06592789,0.09033467,0.02834041,0.2492070,0.04565687,0.02963890,0.03044948
331x498,0.3066188,0.08020827,0.04328161,0.03123564,0.06431247,0.09081447,0.02852401,0.2493011,0.04531932,0.02976247,0.03062183
342x498,0.3145174,0.08153619,0.04218346,0.03250040,0.06160039,0.08904910,0.02800083,0.2456084,0.04404146,0.03057865,0.03038368
320x509,0.3073114,0.08025212,0.04320391,0.03121887,0.06395817,0.09076415,0.02854638,0.2491347,0.04522074,0.02974240,0.03064719


,x,y
,<dbl>,<dbl>
331x476,331,476
331x487,331,487
320x498,320,498
331x498,331,498
342x498,342,498
320x509,320,509


In [11]:
# save the high res file
# head(location_imputation)

props.df.refined <- as.data.frame(CARD_obj@refined_prop)

outDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDOuputs/"
outpath = glue("{outDir}{key}_{subId}CARD_refined.csv")

write.csv(props.df, 
          outpath, 
          row.names=TRUE)

dim(props.df.refined)
head(props.df.refined)

[1] 5566   11

,Mac5,Monocytes,T cells,Mac4,Mac1,NK cells,Dendritic cells,Stromal cells,B cells,Mac2,Mac3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
331x476,0.3034526,0.07928022,0.04397935,0.03105100,0.06866088,0.09012360,0.02800539,0.2493845,0.04629264,0.02963406,0.03013585
331x487,0.3052533,0.07976759,0.04360004,0.03112132,0.06632515,0.09038673,0.02831381,0.2493849,0.04576633,0.02966085,0.03041997
320x498,0.3060100,0.07981767,0.04351404,0.03110309,0.06592789,0.09033467,0.02834041,0.2492070,0.04565687,0.02963890,0.03044948
331x498,0.3066188,0.08020827,0.04328161,0.03123564,0.06431247,0.09081447,0.02852401,0.2493011,0.04531932,0.02976247,0.03062183
342x498,0.3145174,0.08153619,0.04218346,0.03250040,0.06160039,0.08904910,0.02800083,0.2456084,0.04404146,0.03057865,0.03038368
320x509,0.3073114,0.08025212,0.04320391,0.03121887,0.06395817,0.09076415,0.02854638,0.2491347,0.04522074,0.02974240,0.03064719


In [12]:
# pp <- CARD.visualize.gene(
# 	spatial_expression = CARD_obj@refined_expression,
# 	spatial_location = location_imputation,
# 	gene.visualize = c("LPL", "RPL4"),
# 	colors = NULL,
# 	NumCols = 2)

# ggsave("cardFigures/genes.png",
#        width = 20, 
#        height = 20, 
#        units = "cm")

# print('done')

In [13]:
# exp.df <- as.data.frame(rowSums(spt))
# exp.df$names <- rownames(exp.df)
# exp.df.sorted <- exp.df[order(exp.df$rowSums, decreasing = TRUE),]
# head(exp.df.sorted, 20)

In [14]:
# ## select the cell type that we are interested
# ct.visualize = unique(meta.df$cellType)

# ## visualize the spatial distribution of the cell type proportion
# p <- CARD.visualize.prop(
# 	proportion = CARD_obj@Proportion_CARD,        
# 	spatial_location = CARD_obj@spatial_location, 
# 	ct.visualize = ct.visualize,
#     colors = c("lightblue","lightyellow","red"),
#     NumCols=4
# )

# ggsave("test.png", width = 20, height = 20, units = "cm")



# # print(p2)